In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression


In [3]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema= True, header= True )

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
for item in data.head(2)[1]:
    
    print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264


In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols = ['Avg Session Length','Time on App','Time on Website','Length of Membership'], outputCol='features')

In [13]:
output = assembler.transform(data)

In [16]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [18]:
final_data = output.select('features', 'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [19]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                359|
|   mean| 498.79746417012797|
| stddev|  81.30990264142184|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                141|
|   mean|  500.6292871788616|
| stddev|  74.25541077086315|
|    min| 256.67058229005585|
|    max|  712.3963268096637|
+-------+-------------------+



In [22]:
lr = LinearRegression(featuresCol="features", labelCol= "Yearly Amount Spent" )

In [23]:
lr_model = lr.fit(train_data)

In [24]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.726293645354019|
|0.05039631854032223|
| -4.074826411252104|
|-3.7277919266238655|
| -7.487922505192955|
|  10.63933189413433|
|  4.141296918790999|
|-3.8977092928028014|
| 3.0269023589841595|
| -5.411297933665651|
| -8.730541668115563|
|-2.0032649639344413|
|  8.598027591670018|
| -5.557329092219959|
| -8.671515643559985|
| 11.449662134682626|
|  8.478775354884306|
| -8.399410938800202|
|  4.690601808292854|
|  5.301588392627195|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

9.581491093009484

In [28]:
test_results.r2

0.9832312354198995

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.3584771924370...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.8186165667690...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9096268275227...|
|[31.9549038566348...|
|[32.0085045178551...|
|[32.0215955013870...|
|[32.0305497162129...|
+--------------------+
only showing top 20 rows



In [31]:
predictions = lr_model.transform(unlabeled_data)

In [32]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...|287.19753936526854|
|[30.5743636841713...|442.01401743952533|
|[30.8364326747734...| 471.5767268382417|
|[30.8794843441274...|493.93439191147854|
|[31.1280900496166...| 564.7406092522476|
|[31.1695067987115...| 416.7171989081585|
|[31.3584771924370...| 491.0346535306844|
|[31.4252268808548...| 534.6644279475647|
|[31.4459724827577...| 481.8500625761444|
|[31.5257524169682...|449.37692474354753|
|[31.5261978982398...| 417.8250678604534|
|[31.8186165667690...| 448.4219383340701|
|[31.8512531286083...| 464.3942190751284|
|[31.8745516945853...|397.84257333848745|
|[31.8854062999117...| 398.7747886160355|
|[31.9096268275227...| 551.9963735385566|
|[31.9549038566348...|431.51910458504267|
|[32.0085045178551...| 451.5966319675556|
|[32.0215955013870...| 516.8815729495345|
|[32.0305497162129...| 588.9728950259846|
+--------------------+------------